In [6]:
import scvelo as scv
import seaborn as sns
import matplotlib.pyplot as plt

# --- 配色和高亮类型 ---
highlight = ['L2_3.IT.CTX.0','L2_3.IT.CTX.1','L2_3.IT.CTX.2']
set2 = sns.color_palette("Set2", n_colors=len(highlight))

# 构造完整调色板：高亮类型用 Set2，其它都用灰色
all_types = l23_PC1_shifted.obs['RCTD'].cat.categories.tolist()
palette = {t: 'lightgray' for t in all_types}
for t, c in zip(highlight, set2):
    palette[t] = c

# --- 行是类型（3），列是切片（4） ---
types = highlight
slices = ['NR1_1','NR2_1','DR1_1','DR2_1']

fig, axs = plt.subplots(
    nrows=len(types),
    ncols=len(slices),
    figsize=(8, 4),
    constrained_layout=True
)

for i, typ in enumerate(types):
    for j, sl in enumerate(slices):
        ax = axs[i, j]
        # 只取当前 slice 数据
        slice_data = l23_PC1_shifted[l23_PC1_shifted.obs['slice']==sl]
        # 画出属于 typ 的细胞，其它细胞自动被当成灰色
        scv.pl.scatter(
            slice_data,
            basis='stereo_rotated',
            color='RCTD',
            groups=[typ],
            palette=palette,
            size=25,
            ax=ax,
            legend_loc='lower center',
            show=False
        )
        ax.axis('off')


#plt.show()
plt.savefig("F:/spatial/RSP/figures/RSP_RCTD.pdf", dpi=300, bbox_inches='tight')
plt.close()


Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).
Renamed 'stereo_rotated' to convention 'X_stereo_rotated' (adata.obsm).


In [123]:
import matplotlib.pyplot as plt
import scvelo as scv
import numpy as np

# 假设 gene_sets 已经定义，并且有 6 个基因集
# NR_A, NR_B, NR_C, DR_A, DR_B, DR_C
# 获取数据：l23_shifted

# 设置切片
slices = ['NR1_1', 'NR2_1', 'DR1_1', 'DR2_1']
slice_gene_sets = {
    'NR1_1': ['NR_0', 'NR_1', 'NR_2'],
    'NR2_1': ['NR_0', 'NR_1', 'NR_2'],
    'DR1_1': ['DR_0', 'DR_1', 'DR_2'],
    'DR2_1': ['DR_0', 'DR_1', 'DR_2']
}

# 创建一个4列3行的子图布局
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(5, 2))
axes = axes.flatten()  # 将二维数组展平，方便索引

# 计数器，用来确定在哪个子图位置绘制
subplot_idx = 0

# 创建一个列表用于保存所有表达量，用于colorbar
all_expression_values = []

# 为了存储所有的表达量，我们需要一个字典
expression_dict = {}

# 收集所有需要绘制的基因表达数据
for col_idx, slice_type in enumerate(slices):
    # 获取对应的基因集
    gene_sets_for_slice = slice_gene_sets[slice_type]
    
    # 获取该切片的数据
    slice_data = l23_shifted[l23_shifted.obs['slice'] == slice_type]
    
    # 为每个基因集绘制散点图
    for row_idx, gene_set_name in enumerate(gene_sets_for_slice):
        # 获取基因集的基因列表
        gene_set = gene_sets[gene_set_name]
        
        # 获取 gene_set 和 slice_data 中的基因的交集
        common_genes = list(set(gene_set) & set(slice_data.var_names))
        
        if len(common_genes) == 0:
            print(f"Warning: No valid genes from {gene_set_name} in {slice_type}")
            continue
        
        # 筛选出交集中的基因
        slice_data_filtered = slice_data[:, common_genes]
        
        # 计算基因集在该切片中的表达值（例如，取平均值或者总和）
        total_expression = slice_data_filtered.X.mean(axis=1).A1  # 每个细胞的平均表达值
        
        # 获取表达值的1%和99%分位数
        vmin = np.percentile(total_expression, 5)
        vmax = np.percentile(total_expression, 95)
        
        # 将小于1%的值设为1%的值，大于99%的值设为99%的值
        total_expression_clipped = np.clip(total_expression, vmin, vmax)
        
        # 不进行标准化，直接使用剪裁后的表达量
        expression_dict[f'{slice_type}_{gene_set_name}'] = total_expression_clipped
        
        # 获取对应的坐标
        stereo_coordinates = slice_data.obsm['stereo_rotated']
        
        # 绘制散点图
        ax = axes[row_idx * 4 + col_idx]  # 计算当前图的位置
        scatter = ax.scatter(stereo_coordinates[:, 0], stereo_coordinates[:, 1], c=total_expression_clipped, cmap='coolwarm', s=1)
        
        #ax.set_title(f'{slice_type} - {gene_set_name}')
        ax.axis('off')  # 隐藏坐标轴
        
        # 保存表达量数据，用于colorbar
        all_expression_values.extend(total_expression_clipped)

# 添加colorbar
#cbar_ax = fig.add_axes([0.1, -0.05, 0.8, 0.02])  # 重新调整colorbar的位置，使其不重叠
#norm = plt.Normalize(vmin=np.percentile(all_expression_values, 5), vmax=np.percentile(all_expression_values, 95))  # 设置colorbar的范围为5%-95%
#cbar = fig.colorbar(plt.cm.ScalarMappable(cmap='coolwarm', norm=norm), cax=cbar_ax, orientation='horizontal')

# 设置colorbar的刻度，只显示起始和终止标签
#cbar.set_ticks([norm.vmin, norm.vmax])  # 设置colorbar的刻度
#cbar.set_ticklabels([f'5%', f'95%'])  # 设置刻度标签为5%和95%

# 手动调整布局，使得图像不与colorbar重叠
plt.subplots_adjust(bottom=0.02)

# 显示结果
#plt.show()
output_path = "F:/spatial/RSP/figures/RSP_012.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()

print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/RSP/figures/RSP_012.pdf


In [82]:
import matplotlib.pyplot as plt
import seaborn as sns

# 假设 filtered 是你的 DataFrame，只包含想画的那几类
groups = ['L2_3.IT.CTX.0', 'L2_3.IT.CTX.1', 'L2_3.IT.CTX.2']
colors = sns.color_palette("Set2", n_colors=len(groups))
filtered = NR_l23_PC1.obs[NR_l23_PC1.obs['RCTD'].isin(groups)]

fig, ax = plt.subplots(figsize=(4,4))
for grp, col in zip(groups, colors):
    sub = filtered[filtered['RCTD'] == grp]
    sns.kdeplot(
        data=sub,
        x="digital_layer",
        color=col,       # ← 单条曲线指定颜色
        label=grp,
        common_norm=False,
        ax=ax
    )

#ax.legend(title="RCTD")


# 去掉上、右边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

#ax.set_xlabel('Digital Layer')
#ax.set_ylabel('Density')
ax.set_title('NR Digital Layer Density by RCTD')

plt.tight_layout()
fig.savefig("F:/spatial/RSP/figures/NR_l23.pdf", format='pdf', bbox_inches='tight')
plt.close(fig)


In [83]:
import matplotlib.pyplot as plt
import seaborn as sns

# 假设 filtered 是你的 DataFrame，只包含想画的那几类
groups = ['L2_3.IT.CTX.0', 'L2_3.IT.CTX.1', 'L2_3.IT.CTX.2']
colors = sns.color_palette("Set2", n_colors=len(groups))
filtered = DR_l23_PC1.obs[DR_l23_PC1.obs['RCTD'].isin(groups)]

fig, ax = plt.subplots(figsize=(4,4))
for grp, col in zip(groups, colors):
    sub = filtered[filtered['RCTD'] == grp]
    sns.kdeplot(
        data=sub,
        x="digital_layer",
        color=col,       # ← 单条曲线指定颜色
        label=grp,
        common_norm=False,
        ax=ax
    )

#ax.legend(title="RCTD")


# 去掉上、右边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

#ax.set_xlabel('Digital Layer')
#ax.set_ylabel('Density')
ax.set_title('DR Digital Layer Density by RCTD')

plt.tight_layout()
fig.savefig("F:/spatial/RSP/figures/DR_l23.pdf", format='pdf', bbox_inches='tight')
plt.close(fig)
